# How to interpret questions in jupyter notebooks?

What is code completion for jupter notebooks?


In [1]:
var fs = require('fs');
var path = require('path');
if (typeof importNotebook == 'undefined') {
    // Does this work everywhere?
    try {
        var m, co = '', vm = require('vm');
        var source = require('fs').readFileSync('How to find answers.ipynb').toString();
        var re = new RegExp('\\s*"cell_type": "code",\\s*"source":\\s*\\[\\s*([\\s\\S]*?\\$\\$.done[\\s\\S]*?)\\s*\\]\\s*,\s*', 'ig');
        while (m = re.exec(source)) { co += (co == '' ? '' : ',\n' ) + m[1] + ',"\\n"'; }
        co = '\n\$$ = get$$();\n\n' + JSON.parse('['+co+']').join('') + '\n\n;';
        global.getGlobal = () => global;
        global.get$$ = () => {
            var temp$$ = Object.assign({}, global.$$);
            return Object.assign(temp$$, {
                async: () => {},
                done: (done) => {}
            });
        };
        var tmpGlobal = Object.assign({}, global);
        var context = vm.createContext(tmpGlobal);
        (new vm.Script(co)).runInContext(context);
        $$.async();
        tmpGlobal.imports.then((done) => {
            Object.assign(global, tmpGlobal.commandRegistry);
            $$.done(done);
        });
    } catch (e) {
        $$.done(e);
    }
}

$$.done('interpreter is ready');


Sucessfully ran a script in it's own context!


'interpreter is ready'

How to find all the notebooks and read markdown cells?

How to get markdown from all notebooks in {directory}?


In [2]:

var flatten = arr => arr.reduce((acc, val) => 
      acc.concat(Array.isArray(val) ? flatten(val) : val), []);

Array.prototype.flatten = function() {return flatten(this)};

function escapeRegExp(str) {
  return str.replace(/[\-\[\]\/\{\}\(\)\*\+\?\.\\\^\$\|]/g, "\\$&");
}

var walkSync = d => {
    var reg = d.split('*').slice(1).join('[^/]*');
    var dir = d.split('*')[0];
    return flatten(fs.readdirSync(dir)
    .filter(file => path.join(dir, file).match(reg))
    .map(file => fs.statSync(path.join(dir, file)).isDirectory()
        ? walkSync(path.join(dir, file))
        : path.join(dir, file).replace(/\\/g, '/')));
};
commandRegistry['walkSync'] = walkSync;
var notebooks = walkSync(path.resolve('**/*.ipynb'));
console.log(notebooks);
$$.done(walkSync);


ReferenceError: commandRegistry is not defined

How to find questions leading up to jupter cells?

This is a complex question to answer.  Perhaps using IBM Watson language alchemy?


In [3]:
// TODO: interpret markdown leading up to code results and find the resulting function in the list
// for now, the boring solution is to assume all markdown output is a question?
if (typeof markdownCache === 'undefined') {
    var markdownCache = {};
}
if (typeof commandRegistry === 'undefined') {
    var commandRegistry = {};
}
var interpretCells = (cells, notebook) => (resolve, reject) => {
    var previous = [];
    var commandsAdded = [];
    var sourceCount = 0;
    cells.forEach((cell) => {
        var source = cell.source.join('').trim();
        if(cell.cell_type == 'markdown') {
            previous[previous.length] = source;
        } else if (cell.cell_type == 'code') {
            if (typeof commandRegistry[notebook+'['+sourceCount+']'] !== 'undefined') {
                markdownCache[notebook+'['+sourceCount+']'] = previous;
                commandsAdded[commandsAdded.length] = notebook+'['+sourceCount+']';
                previous = [];
            }
            sourceCount++;
        }
    });
    resolve(commandsAdded);
};
var cells = [
    {cell_type: 'code', source: []},
    {cell_type: 'markdown', source: ["test for errors"]},
    {cell_type: 'code', source: ["(function (err) { throw error; })"]}
];
commandRegistry['How to test import[1]'] = cells[2].source.join('').trim();
interpretCells(cells, 'How to test import')(() => {}, () => {});
if(markdownCache['How to test import[1]'][0] == cells[1].source) {
    commandRegistry['interpretCells'] = interpretCells;
    $$.done(interpretCells);
}



started notebook How to find answers.ipynb with 9 cells
Sucessfully ran a script in it's own context!
Already imported How to find answers.ipynb
undefined
[ 'import ready',
  [Function: getCells],
  [Function: promiseContext],
  [Function: runAllPromises],
  [Function: importNotebook],
  undefined,
  undefined,
  'done importing importing',
  undefined ]
found function How to find answers.ipynb[1]
found function How to find answers.ipynb[2]
found function How to find answers.ipynb[3]
found function How to find answers.ipynb[4]
{ 'How to find answers.ipynb': 
   { 'How to find answers.ipynb[1]': [Function: getCells],
     'How to find answers.ipynb[2]': [Function: promiseContext],
     'How to find answers.ipynb[3]': [Function: runAllPromises],
     'How to find answers.ipynb[4]': [Function: importNotebook] } }
{ 'How to find answers.ipynb[1]': [Function: getCells],
  'How to find answers.ipynb[2]': [Function: promiseContext],
  'How to find answers.ipynb[3]': [Function: runAllPromises]

{ 'How to find answers.ipynb[1]': [Function: getCells],
  'How to find answers.ipynb[2]': [Function: promiseContext],
  'How to find answers.ipynb[3]': [Function: runAllPromises],
  'How to find answers.ipynb[4]': [Function: importNotebook] }

[Function: interpretCells]

How to interpret a jupyter {directory}?

In [4]:
var fs = require('fs');
if(typeof commandRegistry === 'undefined') {
    commandRegistry = {};
}
if(typeof markdownCache === 'undefined') {
    markdownCache = {};
}
try {
    var interpretDirectory = (dirname, onFileContent, onError) => {
        var filenames = walkSync(dirname);
        var imports = filenames.map(notebook => {
            return importNotebook(notebook).then(() => notebook);
        });
        var cells = Promise.all(imports)
            .then(notebooks => notebooks.map(notebook => {
                return getCells(notebook, ['markdown', 'code'])
                        .then(cells => interpretCells(cells, notebook));
            }));
        return cells.then(promises => Promise.all(promises))
            .then(promises => runAllPromises(promises));
    }
    commandRegistry['interpretDirectory'] = interpretDirectory;
    
    // test to make sure it works
    $$.async();
    interpretDirectory(path.resolve('**/How to interpret questions.ipynb'))
        .then((results) => {
            console.log(Object.keys(markdownCache));
            $$.done(interpretDirectory);
        }).catch(e => $$.done(e));
}catch(e) {
    $$.done(e);
}


started notebook /Users/briancullinan/jupytangular2/notebooks/How to interpret questions.ipynb with 6 cells
[ '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/2017 Update-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/ECommerce-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/Good UX Intro-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/Landing Pages-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/Selenium Automation-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/Untitled-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/Untitled1-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/What is Angular 2-checkpoint.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/.ipynb_checkpoints/Wireframing-checkpoint.ipynb'

In [14]:
if (typeof interpret == 'undefined') {
    require('child_process').execSync('npm install fuse.js');
}
if(typeof markdownCache === 'undefined') {
    markdownCache = {};
}
var fuse;
//console.log(markdownCache);
try {
    var interpret = function (message) {
        var headings = Object.keys(markdownCache).map((k) => ({
            cell: k,
            content: markdownCache[k].join('')
        }));
        var Fuse = require('fuse.js');
        fuse = new Fuse(headings, {shouldSort: true, keys: ['content'], 'id': 'cell'});
        return fuse.search(message);
    };
    var results = interpret('test for errors');
    console.log(markdownCache[results[0]]);
    console.log(commandRegistry[results[0]]);
    commandRegistry['interpret'] = interpret;
    $$.done(interpret);
}
catch (e) {
    console.log(e);
}



[ 'test for errors' ]
(function (err) { throw error; })


[Function: interpret]